# Анализ и проверка модели

## 1. Алгоритмы тестов Jarque-Bera и Колмогорова-Смирнова

In [9]:
from sklearn.linear_model import LinearRegression
import numpy as np
import scipy.stats as stats
from sklearn.metrics import r2_score

import statsmodels.api as sm

# Генерация выборки
sample = np.random.normal(0, 1, 1000)

# Jarque-Bera test
jb_stat, jb_p_value = stats.jarque_bera(sample)

# Колмогорова-Смирнова test
ks_stat, ks_p_value = stats.kstest(sample, 'norm')

jb_stat, jb_p_value, ks_stat, ks_p_value


(0.004203828835045527,
 0.9979002930576744,
 0.025661412907982073,
 0.5172315289653043)

## 2. Проверка остатков модели на нормальность

In [3]:
def helwig_test(data):
    n = len(data)
    z = (data.mean() - 0) / (data.std(ddof=1) / np.sqrt(n))
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    return z, p_value


# Шапиро-Вилька test
shapiro_stat, shapiro_p_value = stats.shapiro(sample)

# Хельвига test (Z-test based)
helwig_stat, helwig_p_value = helwig_test(sample)

shapiro_stat, shapiro_p_value, helwig_stat, helwig_p_value


(0.9988923668861389,
 0.8128262162208557,
 0.24373654952305981,
 0.8074348644338392)

## 3. Теоретические условия применения


### Тест Jarque-Bera
- Подходит для больших выборок.

### Тест Колмогорова-Смирнова
- Не требует, чтобы выборка была большой. Данные должны быть непрерывными.

### Тест Шапиро-Вилька
- Лучше всего использовать для выборок размером менее 50 элементов.

### Тест Хельвига
- Подходит для малых выборок.


## 4. Проверка модели на адекватность

In [15]:
def durbin_watson_test(residuals):
    num = np.sum(np.diff(residuals) ** 2)
    den = np.sum(residuals ** 2)
    return num / den
def breusch_pagan_test(y, x):
    import statsmodels.api as sm
    from statsmodels.stats.diagnostic import het_breuschpagan
    
    y = np.asarray(y).flatten()
    X = sm.add_constant(x)
    residuals = y - sm.OLS(y, X).fit().predict(X)
    bp_test = het_breuschpagan(residuals, X)
    return bp_test



# Примерные данные для регрессии
X = np.random.rand(100, 1) * 10
y = 2.5 * X + np.random.randn(100, 1) * 2

# Обучение модели линейной регрессии
model = LinearRegression().fit(X, y)
y_pred = model.predict(X)

# Расчет R^2
r2 = r2_score(y, y_pred)

# Durbin-Watson test (автокорреляция)
dw_stat = durbin_watson_test(y - y_pred)

# Breusch-Pagan test (гетероскедастичность)
bp_stat, bp_p_value, _, _ = breusch_pagan_test(y, X)

r2, dw_stat, bp_stat, bp_p_value


(0.9329493693443762, 0.0, 1.45663570778356, 0.22746617429889865)